In [1]:
from Models.models import LLMModel
from langchain_core.messages import (HumanMessage)
import json
from Models.schemas import AspectorEvaluatorInputSchema, FeedbackISC, FeedbackBasic,  QualityAspect, AspectorRole

In [3]:

#prompt and generation count
model1 = LLMModel(
    provider="openai_api",
    model="gpt-4-1106-preview",
    output_schema=FeedbackBasic,
    )


model2 = LLMModel(
    provider="fireworks_api",
    model="llama-v3-70b-instruct",
    output_schema=FeedbackBasic,
    )


model3 = LLMModel(
    provider="anthropic_api",
    model="claude-3-opus-20240229",
    )


In [4]:
this_model = model3

messages={
    "messages": [HumanMessage(content="Who is the president of Nigeria")]
    
}
response = this_model(messages)
print(response)

print("--------------------------------")


messages=[HumanMessage(content="What was my first message."),]
response = this_model(messages)
print(response)


print("--------------------------------")



for message in this_model.chat_history:
    print(message)
   
    




INFO: First Message
DEBUG:  content='Here is the response in the specified JSON format:\n\n{\n  "response": "The current President of Nigeria is Muhammadu Buhari. He assumed office on May 29, 2015 after winning the general election as the candidate of the All Progressives Congress party. Prior to serving as president, Buhari was a major general in the Nigerian Army and served as the head of state from 1983 to 1985 after taking power in a military coup."\n}' response_metadata={'id': 'msg_01U5dCvSTrVM19ZsWPwRayDw', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 183, 'output_tokens': 108}} id='run-7faa63df-d5c0-4ce0-81bc-973444fc930d-0'
{'response': 'The current President of Nigeria is Muhammadu Buhari. He assumed office on May 29, 2015 after winning the general election as the candidate of the All Progressives Congress party. Prior to serving as president, Buhari was a major general in the Nigerian Army and served as the hea

In [5]:
this_model.get_total_tokens()

DEBUG: ...API token count


({'in': 483, 'out': 134},
 {'in': ['The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"response": {"description": "The response string returned by the LLM", "title": "Response", "type": "string"}}, "required": ["response"]}\n```Who is the president of Nigeria',
   'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a w

#### LLM AS EVALUATOR


In [6]:

    
# Example usage of LLMasEvaluator
evaluator = LLMModel(
    provider="fireworks_api",
    model="llama-v3-70b-instruct",
    output_schema=FeedbackISC,
    input_schema=AspectorEvaluatorInputSchema,
    name="aspect_evaluator",
    as_evaluator = True,
    try_to_parse=True
)

convo = [
    "user: Hello, are you doing today?",
    "assistant: I'm doing well, thank you for asking! How can I assist you today?",
    "user: I was hoping you could provide some feedback, on a short essay. I wrote. Would you be able to take a look and let me know your thoughts on the grammar and structure?",
    "assistant: Absolutely, I'd be happy to review, your essay and provide feedback on the grammar and overall structure. Please go ahead and send over the essay whenever you're ready, and I'll take a look. Let me know if there are any specific areas you'd like me to focus on in addition to the grammar and structure."
    ]


quality_aspect = QualityAspect(
    name="grammar",
    instruction="The quality of the grammar including how well it is structured and punctuated"
)


# Create an instance of AspectorEvaluatorInputSchema
evaluation_task = AspectorEvaluatorInputSchema(
    quality_aspect=quality_aspect,
    role=AspectorRole.USER_AND_ASSISTANT,
    conversation=convo,
    metadata = {}

)


# Perform evaluation
evaluation_result = evaluator(evaluation_task)
print(evaluation_result)
print(type(evaluation_result))
# print(f"score: {evaluation_result["score"]}")
# print(f"issues: {evaluation_result["issues"]}")
# print(f"comment: {evaluation_result["comment"]}")



INFO: First Message
DEBUG:  content='{\n"issues": [\n"Punctuation error in user\'s first message: missing question mark",\n"Comma splice in assistant\'s first response: \'I\'m doing well, thank you for asking!\' and \'How can I assist you today?\' should be separate sentences"\n],\n"score": 4,\n"comment": "The conversation is well-structured and easy to follow, but there are some minor errors in punctuation and grammar. The assistant\'s responses are clear and polite, but could benefit from more precise grammar."\n} \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\

In [7]:
evaluator.get_total_tokens()

DEBUG: ...API token count


({'in': 533, 'out': 2048},
 {'in': ['You are a world class Evaluator, whose job is to evaluate the quality of the given aspect(s):\nname=\'grammar\' instruction=\'The quality of the grammar including how well it is structured and punctuated\'.\n\nYou are to focus on only the  responses from user and assistant in the conversation. i.e it is crucial that you evaluate the user and assistant messages/responses only.\n\nThink step by step to figure out the correct evaluation(s) and provide you final response in this given format:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"descript

### Thursday 9th of May 2024 Updates 


1. can now accept either list of lang-chain messages or a dict with template variables
2. return only content (json or string depending on the value of try_to_parse)
3. when system message does not contains "format_instructions" placeholder, it is added (so its optional).
4. inputs are passed as MessagePlaceholder with a variable_input = "messages" when you use dict or list of messages
5. You can add more inputs in your promptTemplate, just ensure make you call input dict with all the required inputs
6. Deafult value sets to FeedbackBasic (however this has no effect if try_to_parse is set to false as no output schema is used)
7. if output schema is set to none, no parsing is done irrespective of the value of try_to_parse


### TO DO
1. fix anthropic multiple system message error - FIXED
2. limited retry feature (pass this with the config parameter as retry (0-5), default will be 3)
3. validate output schema when try_to_parse is True (and retry when fails)
4. Prompt Token (output and input) - (This requires adding up all the tokens in each message in the history) - FIXED

In [10]:
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field, validator
from Models.models import LLMModel
from langchain_core.messages import (HumanMessage)
import json
from Models.schemas import AspectorEvaluatorInputSchema, FeedbackISC, FeedbackBasic,  QualityAspect, AspectorRole

class Joke(BaseModel):
    """Joke model"""
    setup: str = Field(..., description="setup for the joke")
    punchline: str = Field(..., description="punchline for the joke")


prompt_template = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "Given this topic={topic}, generate a joke."
                ),
                (
                    "human",
                    "Also, make your output all upper case LIKE THIS."
                )
               
            ]
        )

In [17]:

model1 = LLMModel(
    provider="openai_api",
    model="gpt-4-1106-preview",
    output_schema=Joke,
    chat_history=[],
    try_to_parse = False,
    prompt_template=prompt_template
    )


In [22]:
inputs = {"topic": "dad jokes"}
response = model1(inputs)
print(response, type(response))

DEBUG:  content='```json\n{\n  "setup": "I TOLD MY WIFE SHE SHOULD EMBRACE HER MISTAKES.",\n  "punchline": "SHE GAVE ME A HUG."\n}\n```' response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 229, 'total_tokens': 274}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-32a38f32-5af6-4476-8bae-86eaba46425f-0'
```json
{
  "setup": "I TOLD MY WIFE SHE SHOULD EMBRACE HER MISTAKES.",
  "punchline": "SHE GAVE ME A HUG."
}
``` <class 'str'>


In [23]:
print(response)

```json
{
  "setup": "I TOLD MY WIFE SHE SHOULD EMBRACE HER MISTAKES.",
  "punchline": "SHE GAVE ME A HUG."
}
```


In [24]:
model1.get_total_tokens()

DEBUG: ...API token count


({'in': 229, 'out': 45},
 {'in': ['\nGiven this topic=dad jokes, generate a joke.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Joke model", "properties": {"setup": {"description": "setup for the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "punchline for the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}\n```Also, make your output all upper case LIKE THIS.'],
  'out': ['```json\n{\n  "setup": "I TOLD MY WIFE SHE SHOULD EMBRACE HER MISTAKES.",\n  "punchline": "SHE GAVE ME A HUG."\n}\n```']})